In [1]:
import os
import tweepy
from dotenv import load_dotenv

import json

#### Cargar llaves de la API
La función `load_dotenv` lee el archivo `.env` e inivializa las variables de ambiente que allí están. Luego se puede acceder a dichas variables de ambiente con el módulo `os`

In [24]:
load_dotenv()

consumer_key = os.environ["API_KEY"]
consumer_secret = os.environ["API_KEY_SECRET"]
access_token = os.environ["ACCESS_TOKEN"]
access_token_secret = os.environ["ACCESS_TOKEN_SECRET"]
bearer_token = os.environ["BEARER_TOKEN"]


client = tweepy.Client(
  bearer_token=bearer_token,
  consumer_key=consumer_key,
  consumer_secret=consumer_secret,
  access_token=access_token,
  access_token_secret=access_token_secret,
  wait_on_rate_limit=True
)

La siguiente función recibe las listas de cuentas y tweets estraídos inicialmente para ir agregando las cuentas y los tweets.

In [25]:
def extraer_cuentas_usuarios(lista_cuentas, lista_tweets, response):
    for i in range(len(response.includes['users'])):
        lista_cuentas.append(response.includes['users'][i])
        lista_tweets.append(response.data[i].text)

    print(len(lista_cuentas))

En esta parte es donde se usa el query para buscar Tweets, y luego usa la función anterior para extraer los Tweets que se encuentran en los responses de la paginación que hace Tweepy

In [26]:
# Search Recent Tweets
# This endpoint/method returns Tweets from the last seven days


query = 'depresión yo diagnosticado'
#query = 'depresión yo tengo'
#query = 'depresión yo tengo -lang:es #depresión #ansiedad'
#query = 'Petro'
#query = 'Saben? A veces me pongo en los zapatos de Gustavo Petro y hasta me compadezco de él'
#query = 'eafit'

# The method returns a Response object, a named tuple with data, includes,
# errors, and meta fields

cuentas_usuarios = []
tweets = []
for response in tweepy.Paginator(client.search_all_tweets,
                                query,
                                tweet_fields=['id', 'author_id', 'text'],
                                user_fields=['id', 'username', 'location'],
                                expansions='author_id',
                                start_time='2021-01-20T00:00:00Z',
                                end_time='2022-01-20T00:00:00Z',
                                max_results=100, limit=1):#.flatten(limit=100):
    extraer_cuentas_usuarios(cuentas_usuarios, tweets, response)



97


In [27]:
print(len(cuentas_usuarios))

97


In [ ]:
print(tweets)

In [ ]:
for usuario in cuentas_usuarios:
    print(usuario.location)

Luego, se verifica que los usuarios sean de Colombia. En caso contrario, no se toman en cuenta

In [21]:
# usuarios = []
# ids_usuarios = []
# for usuario in cuentas_usuarios:
#     if (usuario.location is not None):
#         #if ('Medellin' in usuario.location) or ('Medellín' in usuario.location) or ('medellin' in usuario.location) or ('medellín' in usuario.location):
#         if ('Colombia' in usuario.location) or ('colombia' in usuario.location):
#             usuarios.append(usuario.username)
#             ids_usuarios.append(usuario.id)

In [29]:
usuarios = []
ids_usuarios = []
for usuario in cuentas_usuarios:
    usuarios.append(usuario.username)
    ids_usuarios.append(usuario.id)

In [ ]:
print(usuarios)
print(ids_usuarios)

Crear el diccionario con el id que identifica a la persona y su correspondiente usuario

In [32]:
diccionario_ids_usuarios = dict(zip(ids_usuarios, usuarios))

#### Extraer tweets de usuarios
Con se extraen los tweets de los usuarios que se conservaron (las personas de Colombia). Sin embargo, se crea esta función previa ya que al momento de extraer retweets, cuando estos son muy largos (más de 240 caracteres) queda cortado el mensaje, por lo que realicé esta función, que va verificando si es retweet y en caso afirmativo extrae su texto completo que se encuenta la parte de `includes`

In [ ]:
def extraer_tweets_usuarios(id, diccionario, response):
    print(response)
    exit()
    if id in list(diccionario.keys()):
        lista_tweets = diccionario[id]
    else:
        lista_tweets = []

    cont = 0
    for i in range(len(response.data)):
        if response.data[i]['referenced_tweets']:
            try:
                lista_tweets.append(response.includes['tweets'][cont].text)
                cont += 1
            except:
                lista_tweets.append(response.data[i].text)

        else:
            lista_tweets.append(response.data[i].text)

    diccionario[id] = lista_tweets

In [2]:
users_tweets = dict()
for id_usuario in ids_usuarios:
    for response in tweepy.Paginator(client.get_users_tweets,
                                    id_usuario,
                                    tweet_fields=['id', 'author_id', 'text'],
                                    user_fields=['id', 'username', 'location'],
                                    expansions='referenced_tweets.id',
                                    #expansions='author_id',
                                    max_results=100, limit=1):#.flatten(limit=100):
        extraer_tweets_usuarios(id_usuario, users_tweets, response)
        break

NameError: name 'ids_usuarios' is not defined

In [ ]:
print(users_tweets.keys())

In [ ]:
print(list(users_tweets.keys())[0])
print(users_tweets[list(users_tweets.keys())[0]])

In [37]:
print(len(users_tweets[list(users_tweets.keys())[0]]))

99


#### Guardar usuarios y Tweets en archivos

In [38]:
with open("tweets.json", "w") as fp:
    json.dump(users_tweets,fp)

In [39]:
with open("users.json", "w") as fp:
    json.dump(diccionario_ids_usuarios,fp)